In [7]:
include("../MyReverseDiff.jl")
include("../MyEmbedding.jl")
include("../MyMlp.jl")
include("../TensorOperations.jl")

using .MyReverseDiff
using .MyMlp
using .TensorOperations
using JLD2
using Printf
using Random
using Flux

KeyError: KeyError: key "AbstractFFTs" not found

In [ ]:
x = randn(Float32, 3, 3, 2)
m = randn(Float32, 3, 1)
g = ones(Float32, size(x, 1), size(x, 2) * size(m, 2), size(x, 3))


#@btime MyReverseDiff.dif_convolution(x, m, g)
#MyReverseDiff.multi_convolution(x,m)
Flux.conv(x,m)

3×3×2 Array{Float32, 3}:
[:, :, 1] =
  1.0557    0.944897   1.04285
 -1.78939  -2.58384   -2.15422
 -1.54792   1.88677    0.565789

[:, :, 2] =
  1.78599   -0.656651   0.850447
 -1.28118   -1.65133    0.604299
 -0.569459   2.18384   -0.206739